In [2]:
import requests
import json
import pandas as pd

import plotly.express as px

In [5]:

def get_response(url, endpoint, payload):
    r = requests.get(f'{url}/{endpoint}/?params={payload}')
    js = r.json()
    return js

params = [
    ('gas_boiler', 'natural gas', 0.9, 0.9, False),
    ('ashp', 'electricity', 1.7, 1.7, 3.5),
    ('vrf', 'electricity', 3.2, 2.5, 5.5)
]

paramlist = [{
    'state': 'NY',
    'climate_zone': '4A',
    'projection_case': 'MidCase',
    'design_areas': [
        {
            'type': 'MediumOffice',
            'area': 155000 - 15000,
            'heating_fuel': a[1],
            'dhw_fuel': a[1],
            'heating_cop': a[2],
            'dhw_cop': a[3],
            'cooling_cop': a[4],
            'ashrae_standard': '90.1-2013'
        },
        {
            'type': 'MediumOffice',
            'area': 15000,
            'heating_fuel': a[1],
            'dhw_fuel': a[1],
            'heating_cop': a[2],
            'dhw_cop': a[3],
            'cooling_cop': a[4],
            'ashrae_standard': '90.1-2007'
        },        
    ]
} for a in params]

projections = []
enduses = []
for i, p in enumerate(paramlist):
    herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'
    localhost = 'http://127.0.0.1:5000'
    endpoint = 'get_projection_from_reference_buildings'
    payload = json.dumps(p)

    r = get_response(localhost, endpoint, payload)
    df = pd.DataFrame(r['emissions_projection'])
    df['name'] = params[i][0]
    enduses.append((params[i][0], r['enduses']))
    projections.append(df)
    
proj_df = pd.concat(projections)
proj_piv = pd.pivot_table(proj_df, columns='name', values='kg_co2_per_sf', index='year')
fig = px.line(
    proj_piv, 
    width=900, 
    height=400, 
    template='simple_white'
    )

fig.update_layout({
    'yaxis': {'title': 'kgco2e/sf/yr'},
    'title': {'text': 'emissions standard (kg CO2e/sf/yr),<br>NYU Rubin Hall Alts (from ref buildings)', 'x': 0.45}
})
fig.show()


enduse_dict = {e[0]: e[1]['enduses_per_sf'] for e in enduses}
enduse_df_list = []
for k, v in enduse_dict.items():
    df = pd.DataFrame(v)
    df['name'] = k
    enduse_df_list.append(df)

enduse_df = pd.concat(enduse_df_list)
enduse_df['cat'] = enduse_df['enduse'] + '-' + enduse_df['subcategory']
enduse_df_piv = pd.pivot_table(enduse_df, columns='name', values='kbtu_per_sf', index='enduse')

fig = px.bar(
    enduse_df_piv.sort_values(by=enduse_df_piv.columns[0], ascending=False), 
    barmode='group', template='simple_white', 
    width=900, 
    height=500, 
    title = 'enduses by case, ref-building generated')
fig.show()


In [16]:
# manual enduses

config = {
    'state': 'MA',
    'projection_case': 'MidCase',
    'area': 215000,
    'enduses': [
        {
            'enduse': 'elec',
            'fuel': 'electricity',
            'kbtu': 7129e3
        },
        {
            'enduse': 'gas',
            'fuel': 'natural gas',
            'kbtu': 3111e3
        },
    ]
}

payload = json.dumps(config)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'

r = requests.get(
    f'{herokuhost}/get_projection_from_manual_enduses/?params={payload}'
)

js = r.json()
df_projection = pd.DataFrame(js['emissions_projection'])
df_enduses = pd.DataFrame(js['enduses']['enduses_per_sf'])
df_enduses
px.pie(df_enduses, values='kbtu', names='fuel', labels='kbtu', width=250, height=250, hole=0.5, title='site energy').show()
px.line(df_projection, x='year', y='kg_co2_per_sf', width=500,height=300).show()

In [17]:
config = {
        'types': [
            {'type': 'B_norm', 'area': 50000},
            {'type': 'A', 'area': 3000},
        ],
        'utilities': [
            {'type': 'electricity', 'val': 950000 * 3.412},  # kbtu
            {'type': 'natural_gas', 'val': 10000 * 100},  # kbtu
        ]
    }


payload = json.dumps(config)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'

r = requests.get(
    f'{herokuhost}/compliance/compile_ll97_summary/?params={payload}'
)

js = r.json()

js

{'emissions_thresholds': [455.22, 239.1, 74.2],
 'annual_building_carbon_tons': 327.6209246,
 'carbon_above_thresholds': [-127.5990754,
  88.52092460000003,
  253.42092460000003],
 'annual_penalties': [0, 23723.60779280001, 67916.80779280001]}

In [18]:
config = {
        'types': [
            {'type': 'office', 'area': 28000},
        ],
        'utilities': [
            {'type': 'electricity', 'val': 860000 * 3.412},  # kbtu
            {'type': 'natural_gas', 'val': 0 * 100},  # kbtu
        ]
    }

payload = json.dumps(config)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'

r = requests.get(
    f'{herokuhost}/compliance/compile_berdo_summary/?params={payload}'
)

js = r.json()

js['acp_payments']


[{'year': 2021, 'val': 0.0},
 {'year': 2022, 'val': 0.0},
 {'year': 2023, 'val': 0.0},
 {'year': 2024, 'val': 0.0},
 {'year': 2025, 'val': 0.0},
 {'year': 2026, 'val': 0.0},
 {'year': 2027, 'val': 0.0},
 {'year': 2028, 'val': 0.0},
 {'year': 2029, 'val': 0.0},
 {'year': 2030, 'val': 21604.71456},
 {'year': 2031, 'val': 20231.452799999995},
 {'year': 2032, 'val': 18858.19104},
 {'year': 2033, 'val': 17484.929280000004},
 {'year': 2034, 'val': 16111.66752},
 {'year': 2035, 'val': 19980.005759999996},
 {'year': 2036, 'val': 18606.744000000002},
 {'year': 2037, 'val': 17233.482240000005},
 {'year': 2038, 'val': 15860.220480000002},
 {'year': 2039, 'val': 14486.958719999999},
 {'year': 2040, 'val': 18355.296960000003},
 {'year': 2041, 'val': 16982.035200000002},
 {'year': 2042, 'val': 14922.14256},
 {'year': 2043, 'val': 13548.880800000003},
 {'year': 2044, 'val': 12175.61904},
 {'year': 2045, 'val': 16043.95728},
 {'year': 2046, 'val': 14670.695520000001},
 {'year': 2047, 'val': 13297.4337

In [36]:


### MULTIPLE CASES


cases = ['MidCase', 'LowWindCost', 'LowRECost', 'LowBatCost', 'HighRECost']

def get_response(url, endpoint, payload):
    r = requests.get(f'{url}/{endpoint}/?params={payload}')
    js = r.json()
    return js

params = [
    ('midcase', 'electricity', 3.2, 2.5, 5.5, 'MidCase'),
    ('lowwindcost', 'electricity', 3.2, 2.5, 5.5, 'LowWindCost'),
    ('lowrecost', 'electricity', 3.2, 2.5, 5.5, 'LowRECost'),
    ('lowbatcost', 'electricity', 3.2, 2.5, 5.5, 'LowBatCost'),
    ('highrecost', 'electricity', 3.2, 2.5, 5.5, 'HighRECost')
]

paramlist = [{
    'state': 'NY',
    'climate_zone': '4A',
    'projection_case': a[5],
    'design_areas': [
        {
            'type': 'MediumOffice',
            'area': 155000 - 15000,
            'heating_fuel': a[1],
            'dhw_fuel': a[1],
            'heating_cop': a[2],
            'dhw_cop': a[3],
            'cooling_cop': a[4],
            'ashrae_standard': '90.1-2013'
        },
        {
            'type': 'MediumOffice',
            'area': 15000,
            'heating_fuel': a[1],
            'dhw_fuel': a[1],
            'heating_cop': a[2],
            'dhw_cop': a[3],
            'cooling_cop': a[4],
            'ashrae_standard': '90.1-2007'
        },        
    ]
} for a in params]


projections = []
enduses = []
for i, p in enumerate(paramlist):
    herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'
    localhost = 'http://127.0.0.1:5000'
    endpoint = 'get_projection_from_reference_buildings'
    payload = json.dumps(p)

    r = get_response(herokuhost, endpoint, payload)
    df = pd.DataFrame(r['emissions_projection'])
    df['name'] = params[i][0]
    enduses.append((params[i][0], r['enduses']))
    projections.append(df)
    



proj_df = pd.concat(projections)
proj_piv = pd.pivot_table(proj_df, columns='name', values='kg_co2_per_sf', index='year')


proj_range = pd.concat([
    proj_piv.min(axis=1),
    proj_piv['midcase'],
    proj_piv.max(axis=1),
], axis=1)
proj_range.columns = 'minimum', 'midcase', 'maximum'

fig = px.line(
    proj_range, 
    width=900, 
    height=400, 
    template='simple_white'
    )

fig.update_layout({
    'yaxis': {'title': 'kgco2e/sf/yr'},
    'title': {'text': 'Case Range', 'x': 0.45}
})
fig.show()

